# Initialization and imports

In [1]:
#Celda exclusiva colab:
#"""
#Instalar dependencias
!pip install transformers adapters langid scikit-optimize mlflow

In [2]:
import os
os.chdir('/content')
!rm -rf VRID_language_proyect
!rm -rf VRID_language_proyect
!git clone --branch dev_Alonso --single-branch https://github.com/jitalo333/VRID_language_proyect
#Moverse a repositorio
os.chdir('/content/VRID_language_proyect/BERT')
#Montar drive
from google.colab import drive
drive.mount('/content/drive')
#"""

Cloning into 'VRID_language_proyect'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 93 (delta 33), reused 80 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (93/93), 1.28 MiB | 8.02 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Mounted at /content/drive


In [3]:
import pandas as pd
import os
from preprocess import preprocess_record
from translate import translator
from encoder import embed_texts, prepare_data

#Train model
import ast
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
from ML_pipeline_skp import get_est_params_dict, run_bayesian_pipeline, eval_model, safe_log_metric
from ML_pipeline_skp import mlflow_ckeckpoint

# 1) Preprocesamiento de los datos

In [ ]:
# 1) Cargar datos
path = ""
filePATH = os.path.join(path, "data_concatenada.xlsx")
df = pd.read_excel(filePATH,
                   usecols=["Código VRID", "Título", "Resumen", "Keywords", "Interdisciplinario", "Transdisciplinario"]) \
       .fillna("")
# 2) Preprocesar los datos
cols = ["Título", "Resumen", "Keywords"]
df[cols] = df[cols].applymap(lambda x: "" if str(x).strip().upper() == "DESCONOCIDO" else str(x).strip())
df["text_for_embedding"] = df.apply(
    lambda r: preprocess_record(r["Título"], r["Resumen"], r["Keywords"]),
    axis=1
)
df.to_excel("data_preprocessed.xlsx", index=False)

# 2) Traducción del texto

In [ ]:
trans = translator()
df["text_for_embedding_translated"] = df.apply(
    lambda r: trans.detect_and_translate(r.get("text_for_embedding", "")),
    axis=1
)
df.to_excel("data_translated.xlsx", index=False)

# 3) Embedding text

In [ ]:
# 1) Preparar textos y etiquetas
texts, labels = prepare_data(df)

# 2) Calcular embeddings
# Parámetros modelo
BASE_MODEL = "allenai/specter2_base"
ADAPTER_NAME = "allenai/specter2"

emb_texts = embed_texts(texts, BASE_MODEL, ADAPTER_NAME)

# 3) Guardar embeddings y etiquetas
df_dataset = pd.DataFrame(columns=["Código VRID", "labels", "embedings"])
df_dataset["Código VRID"] = df.loc[labels.index, "Código VRID"]
df_dataset["labels"] = labels
df_dataset["embedings"] = emb_texts.tolist()
df_dataset.to_excel("dataset_embed_translated.xlsx", index=False)

# 4) Train classifier

In [4]:
path = "/content/drive/MyDrive/VRID_NLP/code/VRID_proyect/"
filePATH = os.path.join(path, "dataset_embed_translated.xlsx")
df = pd.read_excel(filePATH)
df['embedings'] = df['embedings'].apply(lambda x: np.array(ast.literal_eval(x)))
df.head()


y = df['labels'].to_numpy()
X = df['embedings'].to_numpy()
X = np.vstack(X)

seed = 7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed, stratify=y)

# 2. Elegir modelos a probar
model_keys = [
    #'LogisticRegression',
    #'DecisionTreeClassifier',
    'RandomForestClassifier',
    #'GradientBoostingClassifier',
    'XGBClassifier',
    #'MLPClassifier',
    #'SVC',
    #'SGDClassifier'
]

# 3. Obtener el diccionario de modelos y parámetros
est_params_dict = get_est_params_dict(model_keys)

print(X_train.shape)
print(X_test.shape)
print("📊 Comienzo:", Counter(y_train))
print("📊 Comienzo:", Counter(y_test))

(867, 768)
(97, 768)
📊 Comienzo: Counter({np.int64(1): 499, np.int64(0): 368})
📊 Comienzo: Counter({np.int64(1): 56, np.int64(0): 41})


In [5]:
# 4. Ejecutar entrenamiento, validación y test con tus funciones
results_val, models_dicc = run_bayesian_pipeline(est_params_dict, X_train, y_train, n_iter=2, sample_weight_On = True)

print(results_val)
for model in models_dicc.values():
  results_test = eval_model(model, X_test, y_test)
  print(results_test)

experiment_name = 'test_functions'
mlflow_ckeckpoint(results_val, models_dicc, X_test, y_test, experiment_name)

RandomForestClassifier
Compute sw
XGBClassifier
Compute sw
{'RandomForestClassifier': {'mean_test_score': 0.83, 'std_test_score': 0.03}, 'XGBClassifier': {'mean_test_score': 0.72, 'std_test_score': 0.04}}
{'accuracy': 0.7010309278350515, 'precision': 0.6956521739130435, 'recall': 0.8571428571428571, 'f1_score': 0.768, 't_n': np.int64(20), 'f_p': np.int64(21), 'f_n': np.int64(8), 't_p': np.int64(48)}
{'accuracy': 0.711340206185567, 'precision': 0.7333333333333333, 'recall': 0.7857142857142857, 'f1_score': 0.7586206896551724, 't_n': np.int64(25), 'f_p': np.int64(16), 'f_n': np.int64(12), 't_p': np.int64(44)}


2025/08/08 17:14:24 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/08/08 17:14:24 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

Registrando modelo en MLflow: RandomForestClassifier
Registrando modelo en MLflow: XGBClassifier
